In [1]:
import cv2
import time
import numpy as np
import contextlib
import torch.cuda.amp
import tensorflow as tf
from sklearn.model_selection import train_test_split

import spconv.pytorch as spconv
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F

from util.eventvision import read_dataset, read_annotation
from util.BoundingBoxRegressionUtils import parse_partial_dataset, parse_random_dataset, torch_iou


TIME_FRAME = 10000

Event-based vision module imported


In [2]:
def getData(filename):
    td = read_dataset(filename)
    
    width = td.width
    height = td.height
    arr = np.array(td.data)
    
    matrix = np.zeros((height, width))
    
    for item in arr:
        if item[3] > TIME_FRAME:
            break
        else:
            x = item[0]
            y = item[1]
            matrix[y][x] = item[3] # item[3] = time stamp
            
    matrix = (matrix / TIME_FRAME)**2       
    matrix = cv2.resize(matrix, (128, 128))
    
    return width, height, matrix

def getAnnotation(filename):
    data = read_annotation(filename)
    minX = min(data[0])
    maxX = max(data[0])
    minY = min(data[1])
    maxY = max(data[1])
    
    return minX, minY, maxX, maxY

In [3]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    amp_ctx = contextlib.nullcontext()
    if args.fp16:
        amp_ctx = torch.cuda.amp.autocast()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        with amp_ctx:
            output = model(data)
            loss = F.mse_loss(output, target)
            loss.backward()
            optimizer.step()

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    amp_ctx = contextlib.nullcontext()
    if args.fp16:
        amp_ctx = torch.cuda.amp.autocast()

    with torch.no_grad():
        for data, target in test_loader:

            data, target = data.to(device), target.to(device)
            with amp_ctx:

                output = model(data)
            test_loss += F.mse_loss(output, target, reduction='sum').item() # sum up batch loss
            
            correct += torch.sum(torch_iou(output[:, -4:], target[:, -4:]))
            

    test_loss /= len(test_loader.dataset)

    
    return correct / len(test_loader.dataset)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.net = spconv.SparseSequential(
            spconv.SparseConv2d(1, 16, 5, stride=1, padding=2),
            nn.LeakyReLU(0.1),
            spconv.SparseMaxPool2d(2, 2),
            
            spconv.SparseConv2d(16, 32, 5, stride=1, padding=2),
            nn.LeakyReLU(0.1),
            spconv.SparseMaxPool2d(2, 2),
            
            spconv.SparseConv2d(32, 64, 5, stride=1, padding=2),
            nn.LeakyReLU(0.1),
            spconv.SparseMaxPool2d(2, 2),
            
            spconv.SparseConv2d(64, 128, 5, stride=1, padding=2),
            nn.LeakyReLU(0.1),
            spconv.SparseMaxPool2d(2, 2),
            
            spconv.SparseConv2d(128, 256, 5, stride=1, padding=2),
            nn.LeakyReLU(0.1),
            spconv.SparseMaxPool2d(2, 2),
            
            spconv.ToDense(),
            nn.Flatten(1,-1),
            nn.Dropout(p=0.2),
            nn.Linear(4096, 2048),
            nn.Linear(2048, 1024),
            nn.Linear(1024, 8),
        )

    def forward(self, x: torch.Tensor):
        x = spconv.SparseConvTensor.from_dense(x.reshape(-1, 128, 128, 1))
        y =  self.net(x)
        
        return y

In [5]:
X_train, X_test, y_train, y_test = parse_partial_dataset(getData, getAnnotation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

train_loader = torch.utils.data.DataLoader(
    [[X_train[i], y_train[i]] for i in range(len(X_train))], shuffle=False, batch_size=40)
# test_loader = torch.utils.data.DataLoader(
#     [[X_test[i], y_test[i]] for i in range(len(X_test))], shuffle=False, batch_size=40)

args = lambda: None
args.fp16 = True
args.lr = 0.001
args.log_interval = 50

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

def makeTestData():
    testData = []
    
    for i in range(10): 
        X_test, y_test = parse_random_dataset(getData, getAnnotation, True)
        test_loader = torch.utils.data.DataLoader(
            [[X_test[i], y_test[i]] for i in range(len(X_test))], shuffle=False, batch_size=40)
        
        testData.append(test_loader)
        
    return testData

def testTiming(model, testData):
    total_time = 0.0
    total_acc = 0.0
    for i in range(10): 
        test_loader = testData[i]

        testing_time = time.time()

        iou = test(args, model, device, test_loader)

        time_taken = time.time()  - testing_time

        total_time += time_taken
        total_acc += iou

        print("\tFor this mini round, time: ", time_taken, " iou ", iou)

    avg_time = total_time / 10
    avg_acc = total_acc / 10
    
    print("Time on avg: ", avg_time, " IOU on avg: ", avg_acc)
    
    return avg_time, avg_acc

def trainModel():
    model = Net().to(device)
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    
    train_start_time = time.process_time()

    for epoch in range(0, 40):
        train(args, model, device, train_loader, optimizer, epoch)

    train_time = time.process_time() - train_start_time
    
    print("Training done!  " + str(train_time))
    
    return train_time, model

def getResults():
    testData = makeTestData()
    
    train_timing_results = np.empty(20)
    test_timing_results = np.empty(20)
    test_acc_results = np.empty(20)

    for i in range(20):
        print("Round ", i)

        train_time, model = trainModel()
        train_timing_results[i] = train_time
        
        test_time, test_acc = testTiming(model, testData)
        test_timing_results[i] = test_time
        test_acc_results[i] = test_acc

    print("\n\nTesting done!\n")

    print("Training time: ", np.mean(train_timing_results))
    print("Testing time: ", np.mean(test_timing_results))
    print("Testing acc: ", np.mean(test_acc_results))
    
    return train_timing_results, test_timing_results, test_acc_results

getResults()

cuda:0
Round  0
Training done!  47.078125
	For this mini round, time:  0.5619997978210449  iou  tensor(0.7546, device='cuda:0')
	For this mini round, time:  0.5510001182556152  iou  tensor(0.7564, device='cuda:0')
	For this mini round, time:  0.5519990921020508  iou  tensor(0.7521, device='cuda:0')
	For this mini round, time:  0.5639991760253906  iou  tensor(0.7588, device='cuda:0')
	For this mini round, time:  0.5460343360900879  iou  tensor(0.7632, device='cuda:0')
	For this mini round, time:  0.5470004081726074  iou  tensor(0.7519, device='cuda:0')
	For this mini round, time:  0.5509998798370361  iou  tensor(0.7496, device='cuda:0')
	For this mini round, time:  0.5570006370544434  iou  tensor(0.7467, device='cuda:0')
	For this mini round, time:  0.5499992370605469  iou  tensor(0.7525, device='cuda:0')
	For this mini round, time:  0.5529990196228027  iou  tensor(0.7565, device='cuda:0')
Time on avg:  0.5533031702041626  IOU on avg:  tensor(0.7542, device='cuda:0')
Round  1
Training d

	For this mini round, time:  0.5450305938720703  iou  tensor(0.7638, device='cuda:0')
	For this mini round, time:  0.5510292053222656  iou  tensor(0.7600, device='cuda:0')
	For this mini round, time:  0.543032169342041  iou  tensor(0.7551, device='cuda:0')
	For this mini round, time:  0.5529994964599609  iou  tensor(0.7625, device='cuda:0')
	For this mini round, time:  0.5560305118560791  iou  tensor(0.7650, device='cuda:0')
Time on avg:  0.5489219188690185  IOU on avg:  tensor(0.7633, device='cuda:0')
Round  9
Training done!  39.515625
	For this mini round, time:  0.5499997138977051  iou  tensor(0.7475, device='cuda:0')
	For this mini round, time:  0.5429883003234863  iou  tensor(0.7531, device='cuda:0')
	For this mini round, time:  0.5500304698944092  iou  tensor(0.7415, device='cuda:0')
	For this mini round, time:  0.5360019207000732  iou  tensor(0.7532, device='cuda:0')
	For this mini round, time:  0.544999361038208  iou  tensor(0.7562, device='cuda:0')
	For this mini round, time: 

Training done!  39.4375
	For this mini round, time:  0.5529682636260986  iou  tensor(0.7444, device='cuda:0')
	For this mini round, time:  0.5360002517700195  iou  tensor(0.7453, device='cuda:0')
	For this mini round, time:  0.5499975681304932  iou  tensor(0.7430, device='cuda:0')
	For this mini round, time:  0.548959493637085  iou  tensor(0.7500, device='cuda:0')
	For this mini round, time:  0.5569884777069092  iou  tensor(0.7500, device='cuda:0')
	For this mini round, time:  0.5520036220550537  iou  tensor(0.7424, device='cuda:0')
	For this mini round, time:  0.5509719848632812  iou  tensor(0.7426, device='cuda:0')
	For this mini round, time:  0.5460171699523926  iou  tensor(0.7355, device='cuda:0')
	For this mini round, time:  0.5469987392425537  iou  tensor(0.7388, device='cuda:0')
	For this mini round, time:  0.5710000991821289  iou  tensor(0.7443, device='cuda:0')
Time on avg:  0.5511905670166015  IOU on avg:  tensor(0.7436, device='cuda:0')
Round  18
Training done!  39.578125
	F

(array([47.078125, 39.78125 , 39.578125, 39.640625, 39.65625 , 39.671875,
        39.390625, 41.      , 39.65625 , 39.515625, 39.5     , 39.546875,
        39.5625  , 39.609375, 39.421875, 39.53125 , 39.28125 , 39.4375  ,
        39.578125, 39.46875 ]),
 array([0.55330317, 0.55190308, 0.54921029, 0.54961634, 0.56059191,
        0.55309675, 0.54920719, 0.55971601, 0.54892192, 0.54729609,
        0.54959819, 0.54850044, 0.54880776, 0.54879851, 0.55060847,
        0.55072203, 0.54640596, 0.55119057, 0.54849424, 0.5499167 ]),
 array([0.75424844, 0.74419296, 0.7373051 , 0.78942674, 0.75631338,
        0.73303252, 0.74555093, 0.74974346, 0.7633301 , 0.7485323 ,
        0.74957383, 0.74972051, 0.75886577, 0.67749876, 0.73999375,
        0.71433794, 0.76086396, 0.74362916, 0.77124232, 0.7497521 ]))